<a href="https://colab.research.google.com/github/MaryGlad/Text-Generator/blob/main/Text_Generation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# libraries

In [1]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=d6b4d72e1d4eba38a2323213e3e2c15cef00bef43ba86101ee89fcc396615cf8
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [2]:
import tensorflow as tf

import numpy as np
import os
import time
import pathlib
import anvil.server

# Making text generator

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pathlib.Path('/content/drive/MyDrive/stories.txt') 

In [ ]:
text = open(data, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))
ids_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_ids(ids), axis=-1)

In [ ]:
all_ids = ids_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
sequences = ids_dataset.batch(101, drop_remainder=True)

In [ ]:
def splt_in_data(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(splt_in_data)

In [ ]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:

vocab_s = len(ids_chars.get_vocabulary())

embed_dim = 256

rnn_un = 1024

In [ ]:
class TextGenModel(tf.keras.Model):
  def __init__(self, vocab_s, embed_dim, rnn_un):
    super().__init__(self)
    
    self.embed = tf.keras.layers.Embedding(vocab_s, embed_dim)
    self.gru = tf.keras.layers.GRU(rnn_un,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_s)

  def call(self, inputs, states=None, return_state=False, train=False):
    x = inputs
    x = self.embed(x, training=train)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=train)
    x = self.dense(x, training=train)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = TextGenModel(
    vocab_s=vocab_s,
    embed_dim=embed_dim,
    rnn_un=rnn_un)

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
checkp_dir = './training_checkpoints'
checkp_pref = os.path.join(checkp_dir, "ckpt_{epoch}")
checkp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkp_pref,
    save_weights_only=True)

In [ ]:
EPOCHS = 200

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkp_callback])

Epoch 1/200
11/11 [==============================] - 88s 7s/step - loss: 4.5936
Epoch 2/200
11/11 [==============================] - 66s 6s/step - loss: 3.5689
Epoch 3/200
11/11 [==============================] - 65s 6s/step - loss: 3.1170
Epoch 4/200
11/11 [==============================] - 65s 6s/step - loss: 2.9594
Epoch 5/200
11/11 [==============================] - 64s 6s/step - loss: 2.7808
Epoch 6/200
11/11 [==============================] - 65s 6s/step - loss: 2.6120
Epoch 7/200
11/11 [==============================] - 64s 6s/step - loss: 2.4992
Epoch 8/200
11/11 [==============================] - 64s 6s/step - loss: 2.4165
Epoch 9/200
11/11 [==============================] - 64s 6s/step - loss: 2.3544
Epoch 10/200
11/11 [==============================] - 64s 6s/step - loss: 2.3038
Epoch 11/200
11/11 [==============================] - 64s 6s/step - loss: 2.2621
Epoch 12/200
11/11 [==============================] - 64s 6s/step - loss: 2.2319
Epoch 13/200
11/11 [=================

In [ ]:
class Generating(tf.keras.Model):
  def __init__(self, model, chars_ids, ids_chars, temperature=0.5):
    super().__init__()

    self.temperature = temperature
    self.model = model
    self.chars_ids = chars_ids
    self.ids_chars = ids_chars

    skip_ids = self.ids_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_chars.get_vocabulary())])
    self.pred_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def gen_one_step(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_chars(input_chars).to_tensor()

    pred_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    pred_logits = pred_logits[:, -1, :]
    pred_logits = pred_logits/self.temperature
    pred_logits = pred_logits + self.pred_mask

    pred_ids = tf.random.categorical(pred_logits, num_samples=1)
    pred_ids = tf.squeeze(pred_ids, axis=-1)

    pred_chars = self.chars_ids(pred_ids)

    return pred_chars, states

In [ ]:
gen_text = Generating(model, chars_ids, ids_chars)

# WEB

In [ ]:
anvil.server.connect("server_FHBYG73CHW6LP5P3525ZB2UM-CVWDV7XBAJZLGH4P")

In [ ]:
@anvil.server.callable
def generate_text(i):
  nxt_char = tf.constant([i])
  result = [nxt_char]
  n_char = 200
  states = None
  for n in range(n_char):
    nxt_char, states = gen_text.gen_one_step(nxt_char, states=states)
    result.append(nxt_char)
  result = tf.strings.join(result)
  return result[0].numpy().decode('utf-8')

In [ ]:
anvil.server.wait_forever()